In [1]:
from src.models.level_kv_div import binaryTrainer, utils, network
from torch.utils.data import DataLoader


import torch

In [2]:
train_dataset = binaryTrainer.get_dataset('../data/processed/sample/train.csv', '../data/processed/sample/train.level.grading')
dev_dataset = binaryTrainer.get_dataset('../data/processed/sample/dev.csv', '../data/processed/sample/dev.level.grading')


KeyboardInterrupt: 

In [ ]:
model = network.SiameseNet(network.EmbeddingNet())
learning_rate = 1e-4
batch_size = 2000
num_epoches = 100

verbose = True
device = 'cuda' if torch.cuda.is_available() else 'cpu'
weight_decay = 1e-6
log_epoch = 3

In [ ]:
# optimizer
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(
        nd in n for nd in no_decay) and p.requires_grad], 'weight_decay': weight_decay},
    {'params': [p for n, p in model.named_parameters() if any(
        nd in n for nd in no_decay) and p.requires_grad], 'weight_decay': 0.0}
]

optimizer = torch.optim.AdamW(params=optimizer_grouped_parameters, lr=learning_rate)

train_loader = DataLoader(train_dataset, shuffle=False, batch_size=batch_size, pin_memory=True)
dev_loader = DataLoader(dev_dataset, shuffle=False, batch_size=batch_size, pin_memory=True)
model = model.to(device)

In [ ]:
best_avg_val_kl_loss, best_model, model = binaryTrainer.train_with_eval(train_loader, dev_loader, model, optimizer, num_epoches, log_epoch, verbose, device)

In [ ]:
best_avg_val_kl_loss